# **Download the dataset**
Link to our datasets :

https://universe.roboflow.com/scripsweet/motif-batik-dga55/dataset/1

https://www.kaggle.com/datasets/alfanme/indonesian-batik-motifs-corak-app/data

We use both of them and combine them so that our model can have more data to train and help increase its accuracy.

**First we need to download the combined version. **

We download it in this link : https://drive.google.com/file/d/1tWiO8DN1IHmImCWz2EVgERROpYSIC5Ot/view?usp=sharing

In [ ]:
# Run this code to download the dataset
!gdown 1tWiO8DN1IHmImCWz2EVgERROpYSIC5Ot

Then we will extract it to the current directory

In [ ]:
# Run this code to extract the file
import zipfile

dataset_zip = './raw_batik_v2.1.zip'
zip_read = zipfile.ZipFile(dataset_zip, 'r')
zip_read.extractall()
zip_read.close()

**Checking the contents of the folder**

In [ ]:
import os

base_dir = './raw_batik_v2.1'

print("Content of base directory:")
print(os.listdir(base_dir))

print("\nContent of train directory:")
print(os.listdir(f'{base_dir}/train'))

print("\nContent of test directory:")
print(os.listdir(f'{base_dir}/test'))

Create variable for shortcut of **train and test path**

In [ ]:
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

Checking total images in every sub-directory of **train**

In [ ]:
for folder in os.listdir(train_dir):
    sub_dir = os.path.join(train_dir, folder)
    if os.path.isdir(sub_dir):  # Check if it's a directory
        # List only image files in the subdirectory
        image_files = [f for f in os.listdir(sub_dir) if os.path.isfile(os.path.join(sub_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]
        print(f'{folder} : {len(image_files)} images')

Checking total images in every sub-directory of **test**

In [ ]:
for folder in os.listdir(test_dir):
    sub_dir = os.path.join(test_dir, folder)
    if os.path.isdir(sub_dir):  # Check if it's a directory
        # List only image files in the subdirectory
        image_files = [f for f in os.listdir(sub_dir) if os.path.isfile(os.path.join(sub_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]
        print(f'{folder} : {len(image_files)} images')

# **Building the Model**

1. Import Library yang Dibutuhkan :
    - tensorflow
    - numpy

In [ ]:
# Tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Additional Library
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

2. Pra-Pemrosesan Data
    Kita akan menggunakan ImageDataGenerator untuk melakukan pra-pemrosesan data dan augmentasi gambar.

In [ ]:
# Pra-Pemrosesan Data
train_datagen = ImageDataGenerator(
                    rescale=1.0/255.0,
                    rotation_range=20,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    # brightness_range=[0.8,1.2],
                    zoom_range=0.6,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
                    rescale=1.0/255.0,)
                    # rotation_range=20,
                    # width_shift_range=0.2,
                    # height_shift_range=0.2,
                    # # brightness_range=[0.8,1.2],
                    # #zoom_range=0.,
                    # horizontal_flip=True,
                    # shear_range = 0.2,
                    # fill_mode = 'nearest')

3. Persiapan Data Latih yang akan Dipelajari oleh Model

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='sparse',
        shuffle=True)

validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='sparse')

## **Creating the model**

## **Transfer learning**

###################################################### Transfer Learning #########################################################################

Import library

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

In [ ]:
# Load model VGG16 sebagai base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Fine-tuning beberapa layer terakhir
for layer in base_model.layers:
    layer.trainable = False

base_model.summary()

In [ ]:
# Mengurangi layers pada PreTrained Model
reduced_pre_trained_model = Model(base_model.input, base_model.layers[-7].output)
reduced_pre_trained_model.summary()

In [ ]:
# Tambahkan layer baru
x = reduced_pre_trained_model.output
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(512, (3,3), activation='relu')(x)
x = tf.keras.layers.Conv2D(512, (3,3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(2,2)(x)
x = tf.keras.layers.Conv2D(1024, (3,3), activation='relu')(x)
x = tf.keras.layers.Conv2D(1024, (3,3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(2,2)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)  # Naikkan nilai dropout
predictions = tf.keras.layers.Dense(25, activation='softmax')(x)

In [ ]:
# Gabungkan base model dan layer baru
model = Model(inputs=reduced_pre_trained_model.input, outputs=predictions)
model.summary()

In [ ]:
# Ubah parameter optimizer
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
opt2 = tf.keras.optimizers.Adam(learning_rate=0.0001)
opt3 = tf.keras.optimizers.SGD(learning_rate=0.001)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt2,
              metrics=['accuracy'])

3. Latih Model

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
# Tambahkan early stopping dan model checkpoint
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
# Latih Model
history = model.fit(
      train_generator,
      steps_per_epoch=10,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=5,
      verbose=2,)
      #callbacks=[early_stopping, model_checkpoint])

Plot akurasi training dan validasi

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

## **Model Prediction**

In [ ]:
# this is if we running on google colab or if we do not have keras-preprocessing module
!pip install keras-preprocessing
!!pip install tensorflowjs

In [ ]:
# save only the model
model.save('model_v2.h5')

# save model with tf js
!tensorflowjs_converter --input_format=keras model_v2.h5 tfjs_model

!zip -r tfjs_model.zip tfjs_model

## Work only on google colab

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import os
from google.colab import files

# Load the trained model
# model = tf.keras.models.load_model('best_model.h5')

# Directory where your test images are located
test_images_dir = os.path.join('./fixed-dataset', 'test')

# Get a list of all the subdirectories (classes)
classes = os.listdir(test_images_dir)

# Function to preprocess the image for prediction
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Resize the image to the target size
    img_array = image.img_to_array(img)  # Convert the image to a numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match the input shape of the model
    img_array /= 255.0  # Normalize the image
    return img_array

# Function to make predictions
def predict_image(img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    return predictions

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
    # predicting images
    path = fn
    predictions = predict_image(path)
    predicted_class = np.argmax(predictions, axis=1)[0]  # Get the class with the highest probability
    predicted_class_name = classes[predicted_class]  # Map index to class name

    # Print the predicted class
    print(f'Image: {path}, Predicted class: {predicted_class_name}')

    # Optionally, plot the image and the predicted class
    img = image.load_img(path, target_size=(224, 224))
    plt.imshow(img)
    plt.title(f'Predicted class: {predicted_class_name}')
    plt.axis('off')
    plt.show()

# **Clean UP**

In [ ]:
import os, signal

# !rm -rf 'raw_batik_v2.1'
# !rm -rf 'raw_batik_v2.1.zip'
os.kill(os.getpid(), signal.SIGKILL)

\